<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/icd10xprice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json as js
import re

In [ ]:
hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')
hospital.columns

<ipython-input-3-d69fde0a6e6e>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')


Index(['ชื่อ', 'รหัส 9 หลักใหม่', 'รหัส 9 หลัก', 'รหัส 5 หลัก',
       'เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก', 'ประเภทองค์กร',
       'ประเภทหน่วยบริการสุขภาพ', 'สังกัด', 'แผนก/กรม', 'ระดับโรงพยาบาล',
       'เตียงที่ใช้จริง', 'สถานะการใช้งาน', 'เขตบริการ', 'ที่อยู่',
       'รหัสจังหวัด', 'จังหวัด', 'รหัสอำเภอ', 'อำเภอ/เขต', 'รหัสตำบล',
       'ตำบล/แขวง', 'หมู่', 'รหัสไปรษณีย์', 'แม่ข่าย', 'วันที่ก่อตั้ง',
       'วันที่ปิดบริการ', 'อัพเดตล่าสุด(เริ่ม 05/09/2566)'],
      dtype='object')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
cols_to_drop = [col for col in data.columns if col.endswith('_txt')]
data = data.drop(columns=cols_to_drop)
df = data[data['claim_status'].isin(['Approve', 'Decline'])]
print(df['claim_status'].value_counts())

claim_status
Approve    15581
Decline      184
Name: count, dtype: int64


In [ ]:
df['hospital_code'] = df['hospital_code'].astype(str)
hospital['hospital_code'] = hospital['รหัส 5 หลัก'].str.replace('="', '').str.replace('"', '')
hospital = hospital[['hospital_code', 'ชื่อ']].rename(columns={'ชื่อ': 'hospital_name'})
df = df.merge(hospital, on='hospital_code', how='left')

<ipython-input-5-83a06c097e2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hospital_code'] = df['hospital_code'].astype(str)


In [ ]:
hospital[hospital['hospital_code'] == '40765']

,hospital_code,hospital_name
1394,40765,โรงพยาบาลทั่วไปขนาดใหญ่เกษมราษฎร์ รัตนาธิเบศร์


In [ ]:
focus = df[df['order_item'] != '[]'][['hospital_code', 'hospital_name', 'cid', 'insurance', 'claim_status','total_bill_amount', 'diagnosis', 'billing', 'order_item', 'response']]

In [ ]:
decline = focus[focus['claim_status'].astype(str)=='Decline']
decline['insurance'].value_counts()

,count
insurance,
tli,23
aia,19
mtl,12
fwd,3


## **❗case: decline**
- **tli**: เข้าข้อยกเว้น, ICD10 อยู่ในกลุ่มโรค

- **aia**: ไม่สามารถเคลมได้ เนื่องจากสัญญาเพิ่มเติมสิ้นผลบังคับ, เบิกซ้ำซ้อนมากกว่า 1 ครั้ง/วัน, ต้องการข้อมูลทางการแพทย์เพิ่มเติม, ผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ

- **mtl**: กรุณาสำรองจ่ายและส่งเอกสารการเคลมให้กับบริษัทฯโดยตรง, โรคมีระยะเวลาการรอคอย 180 วัน, ส่ง fax claim, เงื่อนไขโรคยกเว้นเฉพาะรายบุคคล, โรคที่เข้าข่ายในเงื่อนไขข้อยกเว้นกรมธรรม์สัญญาสุขภาพ

- **fwd**: copay == ผู้ป่วยต้องจ่ายเอง

In [ ]:
js.loads(decline[decline['insurance']=='aia']['response'].iloc[2])

{'detail': [],
 'claim_no': 'C029086468',
 'insurer_detail': [{'claim_no': 'C029086468',
   'non_cover': '',
   'benefit_code': 'HSBypass',
   'copay_amount': '',
   'coverage_code': 'H&S Extra (new standard) ไม่สามารถใช้บริการเรียกร้องสินไหมได้เนื่องจากจำเป็นต้องพิจารณาข้อมูลทางการแพทย์เพิ่มเติม หากต้องการใช้สิทธิ์ กรุณาตรวจสอบความคุ้มครองโดยเจ้าหน้าที่ AIA ( เวลาทำการ 08.30 - 19.00 น.)',
   'excess_amount': '',
   'policy_number': 'T232375422',
   'insurance_type': '',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': 'กรุณาตรวจสอบความคุ้มครองที่เมนู Claim status เวลาทำการ 08.30 – 19.00'}],
 'transaction_no': '3cf7edde-7a78-4ecc-b1f5-aa57b6889514',
 'final_pss_score': '0',
 'transaction_uid': '865787db-8337-47fb-9a9d-4e1b4b94050d',
 'total_excess_amount': '1486.70',
 'total_approved_amount': '0.00'}

In [ ]:
focus[focus['cid']=='1103800114329']['order_item'].iloc[0]

In [ ]:
approve = focus[focus['claim_status'].astype(str)=='Approve']
approve['insurance'].value_counts()

,count
insurance,
mtl,2068
aia,1572
fwd,868
tli,265


เพื่อค้นหา `order_item` ที่เป็นรายการที่พบร่วมกันบ่อยในผู้ป่วยที่เป็นโรคเดียวกัน **และหาค่าราคากลาง**ของ `order_item` นั้น

In [ ]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'K300' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['simb_billing_code', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ simb_billing_code, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary

,simb_billing_code,item_name,average_initial_price,order_count
0,1.1.1(3),ANTACIL GEL (LOCAL) 240 ML (ก),163.0,2
1,1.1.1(3),Alum Milk (Belcid) S,190.0,1
2,1.1.1(3),Antacil gel 240 ml,171.5,2
3,1.1.1(3),Buscopan(L) 20 mg/ml inj.,217.0,1
4,1.1.1(3),DOMPERIDONE 10 MG TAB.,113.0,1
5,1.1.1(3),DOMPERIDONE 10 MG.TAB..,169.0,1
6,1.1.1(3),DOMPERIDONE TAB.,630.0,1
7,1.1.1(3),FAMOTIDINE 20 MG.TAB..,343.0,1
8,1.1.1(3),GASLON N OD 4 MG.TAB..,635.5,2
9,1.1.1(3),Gaszym TAB.,70.0,1


In [ ]:
import json

# แปลง JSON string ให้เป็น dictionary หรือ list
focus['diagnosis'] = focus['diagnosis'].apply(lambda x: json.loads(x))
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))

In [ ]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J069' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['simb_billing_code', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ simb_billing_code, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary


,simb_billing_code,item_name,average_initial_price,order_count
0,1.1.1(3),0,898.000000,1
1,1.1.1(3),0.05 % ILIADIN Nasal,230.000000,4
2,1.1.1(3),0.05%==ผู้ใหญ่==Iliadin nasal drops # (มากกว่...,172.333333,3
3,1.1.1(3),0.3 %D THROAT SPRAY สีเหลือง ( > 6 ปี ) ( LA...,322.733333,15
4,1.1.1(3),10 MG loraTADINE TAB. (LASA),148.000000,3
...,...,...,...,...
264,1.2.1,ตรวจรักษากรณีผู้ป่วยนอก ครั้งแรก (First Out-pa...,500.000000,1
265,1.2.1(1),DF แพทย์ตรวจรักษา (Doctor Fee),731.478873,142
266,1.2.1(1),ค่าตรวจรักษากรณีผู้ป่วยนอก,700.000000,1
267,1.2.1(1),ตรวจรักษาผู้ป่วยนอก-ทั่วไป ใน/นอกเวลา(FU2).,400.000000,1


In [ ]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J029' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['simb_billing_code', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ simb_billing_code, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary


,simb_billing_code,item_name,average_initial_price,order_count
0,1.1.1(3),**Celebrex 200 mg cap.,252.000000,1
1,1.1.1(3),**MYBACIN MOUTHSPRAY 15 ML. (แทนKAMILOSAN,230.000000,1
2,1.1.1(3),**PARACETAMOL 500 MG TAB (TYLENOL),120.000000,1
3,1.1.1(3),0.05 % ILIADIN Nasal,230.000000,1
4,1.1.1(3),0.3 %D THROAT SPRAY สีเหลือง ( > 6 ปี ) ( LA...,335.625000,8
...,...,...,...,...
275,1.2.1(1),ตรวจรักษาผู้ป่วยนอก-ทั่วไป ใน/นอกเวลา(FU2).,400.000000,4
276,1.2.1(11),CHEST PA UPRIGHT (CXR),159.000000,1
277,1.2.1(4),DF แพทย์ตรวจรักษา(ER),700.000000,1
278,1.2.2,ค่าทำศัลยกรรมและหัตถการต่างๆของผู้ประกอบวิชาชี...,454.545455,11


In [ ]:
import json

# กรองข้อมูลที่เป็นโรคที่ต้องการ เช่น "K300"
filtered_data = focus[
    focus['diagnosis'].apply(lambda x: any(d.get('icd10') == 'J000' for d in x))
]

# สร้าง list เก็บ order_item ที่เกี่ยวข้องกับโรคที่กรองได้
order_items = []
for items in filtered_data['order_item']:
    for item in items:
        order_items.append(item)

# แปลง order_items เป็น DataFrame เพื่อช่วยในการคำนวณ
order_items_df = pd.DataFrame(order_items)

# แปลง initial เป็น float ใน order_items_df
order_items_df['initial'] = order_items_df['initial'].astype(float)

# คำนวณค่าราคากลาง (mean) ของแต่ละ simb_billing_code, รวม item_name และจำนวนการสั่ง (count)
order_item_summary = order_items_df.groupby(['simb_billing_code', 'item_name']).agg(
    average_initial_price=('initial', 'mean'),
    order_count=('initial', 'count')
).reset_index()

# แสดงผลรายการ simb_billing_code, item_name, ราคากลาง และจำนวนการสั่ง
order_item_summary


,simb_billing_code,item_name,average_initial_price,order_count
0,1.1.1(3),0.05 % ILIADIN Nasal,230.0,3
1,1.1.1(3),0.05%==ผู้ใหญ่==Iliadin nasal drops # (มากกว่...,203.0,2
2,1.1.1(3),0.3 %D THROAT SPRAY สีเหลือง ( > 6 ปี ) ( LA...,365.0,2
3,1.1.1(3),1 MG **LORA**zepam TAB. (LASA) (ว4),120.0,1
4,1.1.1(3),10 MG loraTADINE TAB. (LASA),104.0,3
...,...,...,...,...
281,1.2.1(1),DF แพทย์ตรวจรักษา(ER),425.0,2
282,1.2.1(1),N/A,400.0,1
283,1.2.1(1),ค่าตรวจรักษากรณีผู้ป่วยนอก,750.0,1
284,1.2.1(1),ค่าแพทย์ GP ตรวจ followและทุก filed ตรวจพรบ.คร...,200.0,1
